In [1]:
import os, sys
import glob
import pandas as pd
import json


PROOT=os.getcwd()

if not os.path.exists("dalton"):
    os.mkdir("dalton")
from madnessToDaltony import *
madmol_files=glob.glob("*.mol")
with open(PROOT+'/molecules/frequency.json') as json_file:
        freq_json = json.loads(json_file.read())

/home/adrianhurtado/testing/madness-test-suite/madness-test-suite


In [2]:
def dalton_excited_input(madmol, xc, basis,num_states):
    molname = madmol.split('.')[0]
    dalton_inp = []
    dalton_inp.append('**DALTON INPUT')
    dalton_inp.append('.RUN PROPERTIES')
    dalton_inp.append('**WAVE FUNCTIONS')
    if xc == 'hf':
        dalton_inp.append('.HF')
    else:
        dalton_inp.append('.DFT')
        dalton_inp.append(xc.capitalize())
    dalton_inp.append('**PROPERTIES')
    dalton_inp.append('.EXCITA')
    dalton_inp.append('*EXCITA')
    dalton_inp.append('.NEXCIT')
    
    states=[]
    for i in range(10):
        states.append(str(num_states))
    dalton_inp.append('   '+'  '.join(states))
        

    dalton_inp.append('**END OF DALTON INPUT')
    dalton_inp = '\n'.join(dalton_inp)
    run_dir = PROOT+"/dalton/"+xc+'/'+molname+'/'+'excited_state'
    if not os.path.exists(run_dir):
        os.makedirs(run_dir)
    madmolfile = PROOT+'/molecules/'+madmol+".mol"
    mol_input = madmol_to_dalmol(madmolfile, basis)
    dal_run_file = run_dir+'/excited.dal'
    with open(dal_run_file, 'w') as file:  # Use file to refer to the file object
        file.write(dalton_inp)
    mol_file = run_dir+"/"+molname+'-'+basis.replace('*', 'S')+'.mol'
    with open(mol_file, 'w') as file:  # Use file to refer to the file object
        file.write(mol_input)
    return run_dir, dal_run_file.split('/')[-1], mol_file.split('/')[-1]

madmol='Be'
xc='hf'
num_states=freq_json[madmol][xc]['excited_state']
run_dir,dal_run_file,mol_file=dalton_excited_input(madmol,xc,'aug-cc-pVDZ',num_states)

In [3]:
from daltonToJson import daltonToJson
madmol='HCl'
xc='hf'
operator='excited_state'
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
def basis_set_calculation(madmol,xc,operator,basis_list):
    output_files=[]
    num_states=freq_json[madmol][xc]['excited_state']
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_excited_input(madmol,xc,basis,num_states)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
        print(output_files[-1])
            
            
    return output_files
        
        
outfiles=basis_set_calculation(madmol,xc,operator,basis_list)

{
  "simulation": {
    "calculations": [
      {
        "calculationType": "energyCalculation",
        "calculationTime": {
          "cpuTime": 0.016,
          "wallTime": 0.017,
          "units": "second"
        },
        "calculationResults": {
          "totalEnergy": {
            "value": -460.092615494889,
            "units": "Hartree"
          },
          "nuclearRepulsionEnergy": {
            "value": 7.057910388957,
            "units": "Hartree"
          },
          "electronEnergy": {
            "value": -467.150525883846,
            "units": "Hartree"
          }
        },
        "calculationSetup": {
          "numberOfElectrons": 18,
          "molecularSpinMultiplicity": 1,
          "waveFunctionType": "HF",
          "waveFunctionTheory": "Hartree-Fock",
          "charge": 0
        }
      },
      {
        "calculationType": "SingletExcitationEnergy",
        "calculationTime": {
          "cpuTime": 0.06,
          "wallTime": 0.06,
          "un

In [4]:
import pandas
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
import numpy as np
#madmol='HCl'
#xc='hf'
#operator='dipole'
# create an energy table

def createExcitedTables(outfiles,basis_list,num_states):
    # generate tables given name of output files and basis_list used to generate output files
    
    data={'totalEnergy':[],'nuclearRepulsionEnergy':[],'electronEnergy':[]}
    rdata={"sym":[],"mode":[],'freq':[]}
    r_dict={}
    s_dict={}
    totals=[]
    for basis,outfile in zip(basis_list,outfiles):
        e_data=json.loads(outfile)['simulation']['calculations'][0]
        r_data=json.loads(outfile)['simulation']['calculations'][1]
        for dkeys in data.keys():
            data[dkeys].append(float(e_data['calculationResults']
                                     [dkeys]['value']))
        freq=r_data['calculationResults']['frequencies']
        freq=[f for l in freq for f in l]
        freq=np.array(freq)
        sort_f=np.argsort(freq)
        freq=freq[sort_f]
        
        # Sort by frequency
        Mode=r_data['calculationResults']['Mode']
        Mode=[m for mo in Mode for m in mo]
        Mode=np.array(Mode)
        Mode=Mode[sort_f]
        Sym=r_data['calculationResults']['Sym']
        Sym=[s for so in Sym for s in so]
        Sym=np.array(Sym)
        Sym=Sym[sort_f]
        
        r_dict['sym']=Sym.tolist()   
        r_dict['mode']=Mode.tolist()  
        r_dict[basis]=freq.tolist()    
        rf=pd.DataFrame.from_dict(r_dict)
        
        s_dict['Sym']=Sym.tolist()   
        s_dict['Mode']=Mode.tolist()  
        s_dict['freq']=freq.tolist()  
        totals.append(pd.DataFrame.from_dict(s_dict))
        
        
    df=pd.DataFrame.from_dict(data,orient='columns')
    return df.T,rf,totals

energy_table,response_table,full_table=createExcitedTables(outfiles,basis_list,4)


In [5]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
madmol='HCl'
xc='hf'
operator='excited-state'
def generateDaltonExcitedDataTable(madmol,xc,num_states,basis_list):
    output_files=[]
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_excited_input(madmol,xc,basis,num_states)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
    
    if not os.path.exists("data_tables"):
        os.mkdir("data_tables")
    
    e,r,ffs=createExcitedTables(output_files,basis_list,num_states)
    e.to_csv('data_tables/'+'e_dalton_'+madmol+'_'+xc+'_'+'excited-state'+'.csv')
    r.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+'excited-state'+'.csv')
    for b,f in zip(basis_list,ffs):
        f.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+'excited-state_'+b+'.csv')
        
        
    return e,r,ffs



In [6]:
e,r,ffs=generateDaltonExcitedDataTable('H2O',xc,4,basis_list)

In [7]:
madmols=glob.glob("molecules/*.mol")
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
xc='hf'
operator='excited-state'
for madmol in madmols:
    madmol=madmol.split('/')[1].split('.')[0]
    print(madmol)
    num_states=freq_json[madmol][xc]['excited_state']
    try:
        generateDaltonExcitedDataTable(madmol,xc,num_states,basis_list)
    except: KeyboardInterrupt
    os.chdir(PROOT)
        

CCl4
CSO
LiH
FCN
P2H4
OCl2
LiH_s
SH2
Li2
NaCl
C2H2
BH3
HCONH2
CH3SH
P2
BH2Cl
NaCN
HCN
CH3OH
HBS
C2H4
SO2
HNC
Na2
F2
CH2PH
N2
SiH4
N2H2
CH3NH2
NH3O
HOF
He
HF
OF2
ClF
HCOOH
NH3
CH3Cl
Mg2
N2H4
Ne
HCHO
PH2OH
SiH3Cl
SF2
SiH3F
AlF
HOCl
HOOH
H2
HCl
HCCF
Mg
CH2BH
CH3BH2
BF
SiO
FNO
H2O
S2H2
LiCl
PH3O
Ar
BHF2
HNS
O3
HBO
Cl2
C6H6



 NOTE:    1 warnings have been issued.
 Check output, result, and error files for "WARNING".


NH2Cl
LiCN
CO
CS
LiBH4



 NOTE:    1 warnings have been issued.
 Check output, result, and error files for "WARNING".


CH4
BeH2
BH2F
CH2NH
NP
CH3F
CO2
PH3
ClCN
HCHS
NaLi
SCl2
Be
NH2OH
HCCCl
NaH
NH2F
HNO
HCP
NOCl


In [12]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
def DaltonOutputToTableExcited(madmol,xc,operator,basis_list,num_states):
    run_path='dalton'
    f_p='/'.join([run_path,xc,madmol,operator])
    output_files=[]
    for basis in basis_list:
        out_path='excited_'+madmol+'-'+basis+'.out'
        full_path='/'.join([f_p,out_path])
        with open(full_path) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
    
    if not os.path.exists("data_tables"):
        os.mkdir("data_tables")
    
    e,r=createExcitedTables(output_files,basis_list)
    e.to_csv('data_tables/'+'e_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    r.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    return e,r
        
for madmol in madmols:
    madmol=madmol.split('/')[1].split('.')[0]
    try:
        DaltonOutputToTableExcited(madmol,xc,operator,basis_list)
    except: FileNotFoundError


        


In [15]:
# get csv file if it exists already
def getDaltonExcitedData(mol,xc,operator):
    dt_path='data_tables/'
    prefix='r_dalton'
    full_path=dt_path+prefix+'_'+mol+'_'+xc+'_'+operator+'.csv'
    print(full_path)
    
    return pd.read_csv(full_path ,index_col=0)

P2=getDaltonExcitedData('P2','hf','excited-state')

data_tables/r_dalton_P2_hf_excited-state.csv


FileNotFoundError: [Errno 2] No such file or directory: 'data_tables/r_dalton_P2_hf_excited-state.csv'